### 실제 활용 시나리오
Q: "로그인 기능 관련 문서 찾아줘" → 하이브리드 검색 → SRS + HLD + TC 모두 반환 + 관계 포함
Q: "SRS-001 변경하면 어디에 영향?" → 영향도 분석 → 설계서 3건, 테스트 2건, 소스 5건 영향
Q: "이 테스트가 어떤 요구사항을 검증하는지?" → 역방향 추적성 → TC-001 ← SRS-001, SRS-002

### 권장 인덱스 구성
- Vector Index: 임베딩 기반 시맨틱 검색
- Inverted Index: 한국어 전문검색 (text_ko analyzer)
- Persistent Index: 필터링 (artifact_type, phase, status)

In [ ]:
"""
SI 프로젝트 산출물 GraphRAG 시스템
- ArangoDB 기반 그래프 + 벡터 하이브리드 검색
- 문서 간 추적성(Traceability) 관리
"""

from dataclasses import dataclass, field
from typing import Optional, Literal
from datetime import datetime
from enum import Enum
import json


# =============================================================================
# 1. SI 산출물 스키마 정의
# =============================================================================

class ArtifactType(str, Enum):
    """SI 프로젝트 산출물 유형"""
    # 분석 단계
    RFP = "RFP"                          # 제안요청서
    PROPOSAL = "PROPOSAL"                # 제안서
    BRD = "BRD"                          # 사업요구사항정의서
    SRS = "SRS"                          # 소프트웨어요구사항명세서
    
    # 설계 단계
    SAD = "SAD"                          # 시스템아키텍처설계서
    HLD = "HLD"                          # 고수준설계서
    LLD = "LLD"                          # 상세설계서
    ERD = "ERD"                          # 데이터모델(ERD)
    API_SPEC = "API_SPEC"                # API 명세서
    UI_SPEC = "UI_SPEC"                  # 화면설계서
    
    # 개발 단계
    SOURCE_CODE = "SOURCE_CODE"          # 소스코드
    DDL = "DDL"                          # DB스크립트
    
    # 테스트 단계
    TEST_PLAN = "TEST_PLAN"              # 테스트계획서
    TEST_CASE = "TEST_CASE"              # 테스트케이스
    TEST_RESULT = "TEST_RESULT"          # 테스트결과서
    
    # 이행/운영
    DEPLOY_GUIDE = "DEPLOY_GUIDE"        # 배포가이드
    USER_MANUAL = "USER_MANUAL"          # 사용자매뉴얼
    OPS_MANUAL = "OPS_MANUAL"            # 운영매뉴얼
    
    # 관리
    WBS = "WBS"                          # 작업분류체계
    MEETING_MINUTES = "MEETING_MINUTES"  # 회의록
    ISSUE = "ISSUE"                      # 이슈
    CHANGE_REQUEST = "CHANGE_REQUEST"    # 변경요청서


class RelationType(str, Enum):
    """산출물 간 관계 유형"""
    DERIVES_FROM = "derives_from"        # A는 B로부터 도출됨
    IMPLEMENTS = "implements"            # A는 B를 구현함
    TESTS = "tests"                      # A는 B를 테스트함
    REFERENCES = "references"            # A는 B를 참조함
    DEPENDS_ON = "depends_on"            # A는 B에 의존함
    SUPERSEDES = "supersedes"            # A는 B를 대체함 (버전)
    RELATED_TO = "related_to"            # 일반적 관련
    ADDRESSES = "addresses"              # A는 B(이슈)를 해결함


@dataclass
class SIArtifact:
    """SI 산출물 노드"""
    _key: str                            # 문서 ID (예: SRS-001)
    artifact_type: ArtifactType
    title: str
    content: str                         # 문서 본문 (청킹된 내용)
    
    # 메타데이터
    project_code: str                    # 프로젝트 코드
    phase: str                           # 단계 (분석/설계/개발/테스트/이행)
    version: str = "1.0"
    status: str = "draft"                # draft, review, approved, obsolete
    
    # 컨텍스트 보강
    keywords: list[str] = field(default_factory=list)
    tags: list[str] = field(default_factory=list)
    related_queries: list[str] = field(default_factory=list)
    summary: Optional[str] = None
    
    # 담당자
    author: Optional[str] = None
    reviewer: Optional[str] = None
    
    # 시간
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    updated_at: str = field(default_factory=lambda: datetime.now().isoformat())
    
    # 벡터 (임베딩)
    embedding: Optional[list[float]] = None
    
    def to_dict(self) -> dict:
        return {
            "_key": self._key,
            "artifact_type": self.artifact_type.value,
            "title": self.title,
            "content": self.content,
            "project_code": self.project_code,
            "phase": self.phase,
            "version": self.version,
            "status": self.status,
            "keywords": self.keywords,
            "tags": self.tags,
            "related_queries": self.related_queries,
            "summary": self.summary,
            "author": self.author,
            "reviewer": self.reviewer,
            "created_at": self.created_at,
            "updated_at": self.updated_at,
            "embedding": self.embedding
        }


@dataclass
class ArtifactRelation:
    """산출물 간 관계 엣지"""
    _from: str                           # 출발 노드 (collection/key)
    _to: str                             # 도착 노드
    relation_type: RelationType
    description: Optional[str] = None
    trace_id: Optional[str] = None       # 추적성 ID
    
    def to_dict(self) -> dict:
        return {
            "_from": self._from,
            "_to": self._to,
            "relation_type": self.relation_type.value,
            "description": self.description,
            "trace_id": self.trace_id
        }


# =============================================================================
# 2. ArangoDB 스키마 및 초기화
# =============================================================================

ARANGODB_SCHEMA = """
// ============================================
// ArangoDB 컬렉션 및 그래프 생성 스크립트
// ============================================

// 1. 문서 컬렉션 생성
db._create("si_artifacts");

// 2. 엣지 컬렉션 생성
db._createEdgeCollection("artifact_relations");

// 3. 그래프 정의
var graph = require("@arangodb/general-graph");
graph._create("si_project_graph", [
    graph._relation("artifact_relations", ["si_artifacts"], ["si_artifacts"])
]);

// 4. 인덱스 생성

// 전문 검색 인덱스 (Analyzer 포함)
db._collection("si_artifacts").ensureIndex({
    type: "inverted",
    name: "content_search_idx",
    fields: [
        { name: "content", analyzer: "text_ko" },
        { name: "title", analyzer: "text_ko" },
        { name: "keywords[*]", analyzer: "identity" },
        { name: "tags[*]", analyzer: "identity" },
        { name: "related_queries[*]", analyzer: "text_ko" }
    ]
});

// 필터링용 인덱스
db._collection("si_artifacts").ensureIndex({
    type: "persistent",
    fields: ["artifact_type", "project_code", "phase", "status"]
});

// 벡터 검색 인덱스 (ArangoDB 3.12+)
db._collection("si_artifacts").ensureIndex({
    type: "vector",
    name: "embedding_idx",
    fields: ["embedding"],
    params: {
        metric: "cosine",
        dimension: 1536,  // OpenAI ada-002 또는 유사 모델
        nLists: 100
    }
});

// 5. 한국어 Analyzer 생성 (필요시)
var analyzers = require("@arangodb/analyzers");
analyzers.save("text_ko", "text", {
    locale: "ko",
    case: "lower",
    accent: false,
    stemming: true,
    stopwords: []
}, ["frequency", "norm", "position"]);
"""


# =============================================================================
# 3. GraphRAG 검색 쿼리들
# =============================================================================

class SIGraphRAGQueries:
    """SI 프로젝트용 GraphRAG 쿼리 모음"""
    
    @staticmethod
    def hybrid_search(
        search_text: str,
        embedding: list[float],
        project_code: str = None,
        artifact_types: list[str] = None,
        phase: str = None,
        limit: int = 10,
        vector_weight: float = 0.6
    ) -> str:
        """
        하이브리드 검색: 벡터 + 전문검색 + 그래프 컨텍스트
        """
        filters = []
        if project_code:
            filters.append(f'doc.project_code == "{project_code}"')
        if artifact_types:
            types_str = ", ".join(f'"{t}"' for t in artifact_types)
            filters.append(f'doc.artifact_type IN [{types_str}]')
        if phase:
            filters.append(f'doc.phase == "{phase}"')
        
        filter_clause = " AND ".join(filters) if filters else "true"
        
        return f"""
        // 1. 벡터 검색 점수
        LET vector_results = (
            FOR doc IN si_artifacts
            FILTER {filter_clause}
            LET vec_score = COSINE_SIMILARITY(doc.embedding, @embedding)
            FILTER vec_score > 0.5
            RETURN {{ doc, vec_score }}
        )
        
        // 2. 전문 검색 점수 (BM25)
        LET text_results = (
            FOR doc IN si_artifacts_view
            SEARCH ANALYZER(
                doc.content IN TOKENS(@search_text, "text_ko") OR
                doc.title IN TOKENS(@search_text, "text_ko") OR
                doc.keywords ANY == @search_text OR
                doc.related_queries ANY IN TOKENS(@search_text, "text_ko"),
                "text_ko"
            )
            FILTER {filter_clause}
            LET text_score = BM25(doc)
            RETURN {{ doc, text_score }}
        )
        
        // 3. 점수 통합 (RRF - Reciprocal Rank Fusion)
        LET combined = (
            FOR vr IN vector_results
            LET tr = FIRST(FOR t IN text_results FILTER t.doc._key == vr.doc._key RETURN t)
            LET hybrid_score = (
                {vector_weight} * vr.vec_score + 
                {1 - vector_weight} * (tr ? tr.text_score / 10 : 0)
            )
            RETURN {{
                doc: vr.doc,
                vec_score: vr.vec_score,
                text_score: tr ? tr.text_score : 0,
                hybrid_score: hybrid_score
            }}
        )
        
        // 4. 그래프 컨텍스트 추가
        FOR result IN combined
        SORT result.hybrid_score DESC
        LIMIT {limit}
        
        LET related = (
            FOR v, e IN 1..2 ANY result.doc._id artifact_relations
            RETURN {{
                artifact: v,
                relation: e.relation_type,
                direction: e._from == result.doc._id ? "outbound" : "inbound"
            }}
        )
        
        RETURN {{
            artifact: result.doc,
            scores: {{
                vector: result.vec_score,
                text: result.text_score,
                hybrid: result.hybrid_score
            }},
            related_artifacts: related
        }}
        """
    
    @staticmethod
    def traceability_query(artifact_key: str, direction: str = "both") -> str:
        """
        추적성 쿼리: 요구사항 → 설계 → 구현 → 테스트 추적
        """
        direction_filter = {
            "forward": "OUTBOUND",   # 요구사항 → 구현
            "backward": "INBOUND",   # 구현 → 요구사항
            "both": "ANY"
        }.get(direction, "ANY")
        
        return f"""
        // 시작 산출물
        LET start_doc = DOCUMENT("si_artifacts/{artifact_key}")
        
        // 추적성 경로 탐색
        FOR v, e, p IN 1..5 {direction_filter} start_doc
        GRAPH "si_project_graph"
        
        // 경로를 추적성 체인으로 구성
        LET trace_chain = (
            FOR i IN 0..LENGTH(p.vertices)-1
            LET vertex = p.vertices[i]
            LET edge = i < LENGTH(p.edges) ? p.edges[i] : null
            RETURN {{
                step: i,
                artifact_type: vertex.artifact_type,
                artifact_key: vertex._key,
                title: vertex.title,
                relation: edge ? edge.relation_type : null
            }}
        )
        
        RETURN DISTINCT {{
            end_artifact: {{
                _key: v._key,
                type: v.artifact_type,
                title: v.title,
                phase: v.phase
            }},
            path_length: LENGTH(p.edges),
            trace_chain: trace_chain
        }}
        """
    
    @staticmethod
    def impact_analysis(artifact_key: str) -> str:
        """
        영향도 분석: 특정 산출물 변경 시 영향받는 문서들
        """
        return f"""
        LET start_doc = DOCUMENT("si_artifacts/{artifact_key}")
        
        // 영향받는 산출물 탐색 (의존 관계 따라가기)
        LET impacted = (
            FOR v, e, p IN 1..10 OUTBOUND start_doc
            GRAPH "si_project_graph"
            OPTIONS {{ uniqueVertices: "global" }}
            
            // 변경 영향이 전파되는 관계만
            FILTER e.relation_type IN ["implements", "derives_from", "depends_on", "tests"]
            
            COLLECT artifact_type = v.artifact_type INTO grouped
            RETURN {{
                artifact_type: artifact_type,
                count: LENGTH(grouped),
                artifacts: grouped[*].v._key
            }}
        )
        
        // 직접 영향
        LET direct_impact = (
            FOR v, e IN 1..1 OUTBOUND start_doc artifact_relations
            FILTER e.relation_type IN ["implements", "derives_from", "depends_on", "tests"]
            RETURN {{
                _key: v._key,
                type: v.artifact_type,
                title: v.title,
                relation: e.relation_type
            }}
        )
        
        RETURN {{
            source: {{
                _key: start_doc._key,
                type: start_doc.artifact_type,
                title: start_doc.title
            }},
            direct_impact: direct_impact,
            impact_summary: impacted,
            total_impacted: SUM(impacted[*].count)
        }}
        """
    
    @staticmethod
    def context_enriched_search(search_text: str, depth: int = 2) -> str:
        """
        컨텍스트 보강 검색: 검색 결과 + 주변 그래프 컨텍스트
        """
        return f"""
        // 1. 기본 검색
        LET search_results = (
            FOR doc IN si_artifacts_view
            SEARCH ANALYZER(
                doc.content IN TOKENS(@search_text, "text_ko") OR
                doc.title IN TOKENS(@search_text, "text_ko") OR
                doc.keywords ANY IN TOKENS(@search_text, "text_ko") OR
                doc.related_queries ANY IN TOKENS(@search_text, "text_ko"),
                "text_ko"
            )
            SORT BM25(doc) DESC
            LIMIT 5
            RETURN doc
        )
        
        // 2. 각 결과에 대해 그래프 컨텍스트 수집
        FOR doc IN search_results
        
        // 상위 문서 (요구사항 등)
        LET upstream = (
            FOR v, e IN 1..{depth} INBOUND doc artifact_relations
            FILTER e.relation_type IN ["implements", "derives_from"]
            RETURN DISTINCT {{
                _key: v._key,
                type: v.artifact_type,
                title: v.title,
                summary: v.summary
            }}
        )
        
        // 하위 문서 (구현, 테스트 등)
        LET downstream = (
            FOR v, e IN 1..{depth} OUTBOUND doc artifact_relations
            FILTER e.relation_type IN ["implements", "tests", "derives_from"]
            RETURN DISTINCT {{
                _key: v._key,
                type: v.artifact_type,
                title: v.title,
                summary: v.summary
            }}
        )
        
        // 관련 이슈/변경요청
        LET issues = (
            FOR v, e IN 1..1 ANY doc artifact_relations
            FILTER v.artifact_type IN ["ISSUE", "CHANGE_REQUEST"]
            RETURN {{
                _key: v._key,
                type: v.artifact_type,
                title: v.title,
                status: v.status
            }}
        )
        
        RETURN {{
            artifact: {{
                _key: doc._key,
                type: doc.artifact_type,
                title: doc.title,
                content: doc.content,
                phase: doc.phase,
                keywords: doc.keywords
            }},
            context: {{
                upstream: upstream,
                downstream: downstream,
                related_issues: issues
            }},
            // LLM 프롬프트용 컨텍스트 문자열 생성
            enriched_context: CONCAT(
                "## ", doc.title, "\\n",
                doc.content, "\\n\\n",
                "### 상위 문서\\n",
                (FOR u IN upstream RETURN CONCAT("- [", u.type, "] ", u.title)),
                "\\n### 하위 문서\\n",
                (FOR d IN downstream RETURN CONCAT("- [", d.type, "] ", d.title))
            )
        }}
        """
    
    @staticmethod
    def project_overview(project_code: str) -> str:
        """
        프로젝트 산출물 현황 개요
        """
        return f"""
        // 프로젝트 산출물 통계
        LET artifacts = (
            FOR doc IN si_artifacts
            FILTER doc.project_code == @project_code
            COLLECT type = doc.artifact_type, phase = doc.phase, status = doc.status
            WITH COUNT INTO cnt
            RETURN {{
                artifact_type: type,
                phase: phase,
                status: status,
                count: cnt
            }}
        )
        
        // 추적성 커버리지
        LET requirements = (
            FOR doc IN si_artifacts
            FILTER doc.project_code == @project_code
            FILTER doc.artifact_type == "SRS"
            
            LET has_design = LENGTH(
                FOR v IN 1..1 OUTBOUND doc artifact_relations
                FILTER v.artifact_type IN ["HLD", "LLD"]
                RETURN 1
            ) > 0
            
            LET has_test = LENGTH(
                FOR v IN 1..3 OUTBOUND doc artifact_relations
                FILTER v.artifact_type == "TEST_CASE"
                RETURN 1
            ) > 0
            
            RETURN {{
                req_key: doc._key,
                title: doc.title,
                has_design: has_design,
                has_test: has_test
            }}
        )
        
        RETURN {{
            project_code: @project_code,
            artifact_summary: artifacts,
            traceability_coverage: {{
                total_requirements: LENGTH(requirements),
                with_design: LENGTH(FOR r IN requirements FILTER r.has_design RETURN 1),
                with_test: LENGTH(FOR r IN requirements FILTER r.has_test RETURN 1)
            }},
            requirements_detail: requirements
        }}
        """


# =============================================================================
# 4. GraphRAG 검색 서비스
# =============================================================================

class SIGraphRAGService:
    """SI 프로젝트 GraphRAG 서비스"""
    
    def __init__(self, db_config: dict):
        """
        db_config: {
            "host": "localhost",
            "port": 8529,
            "username": "root",
            "password": "password",
            "database": "si_project_db"
        }
        """
        self.config = db_config
        self.db = None
        # 실제 구현시 python-arango 사용
        # from arango import ArangoClient
        # self.client = ArangoClient(hosts=f"http://{db_config['host']}:{db_config['port']}")
        # self.db = self.client.db(db_config['database'], ...)
    
    def search(
        self,
        query: str,
        search_type: Literal["hybrid", "trace", "impact", "context"] = "hybrid",
        **kwargs
    ) -> dict:
        """통합 검색 인터페이스"""
        
        if search_type == "hybrid":
            # 임베딩 생성 필요
            embedding = self._get_embedding(query)
            aql = SIGraphRAGQueries.hybrid_search(
                search_text=query,
                embedding=embedding,
                **kwargs
            )
        elif search_type == "trace":
            aql = SIGraphRAGQueries.traceability_query(
                artifact_key=kwargs.get("artifact_key"),
                direction=kwargs.get("direction", "both")
            )
        elif search_type == "impact":
            aql = SIGraphRAGQueries.impact_analysis(
                artifact_key=kwargs.get("artifact_key")
            )
        elif search_type == "context":
            aql = SIGraphRAGQueries.context_enriched_search(
                search_text=query,
                depth=kwargs.get("depth", 2)
            )
        
        # 실제 구현시
        # return self.db.aql.execute(aql, bind_vars={...})
        return {"query": aql, "params": kwargs}
    
    def _get_embedding(self, text: str) -> list[float]:
        """텍스트 임베딩 생성 (실제 구현 필요)"""
        # OpenAI, Sentence-Transformers 등 사용
        return [0.0] * 1536  # placeholder
    
    def add_artifact(self, artifact: SIArtifact) -> str:
        """산출물 추가"""
        # 임베딩 생성
        searchable_text = f"{artifact.title} {artifact.content} {' '.join(artifact.keywords)}"
        artifact.embedding = self._get_embedding(searchable_text)
        
        # 실제 구현시
        # return self.db.collection("si_artifacts").insert(artifact.to_dict())
        return artifact._key
    
    def add_relation(self, relation: ArtifactRelation) -> str:
        """관계 추가"""
        # 실제 구현시
        # return self.db.collection("artifact_relations").insert(relation.to_dict())
        return f"{relation._from}->{relation._to}"


# =============================================================================
# 5. 컨텍스트 보강 유틸리티
# =============================================================================

class ArtifactEnricher:
    """산출물 컨텍스트 보강"""
    
    # 산출물 유형별 예상 질의 템플릿
    QUERY_TEMPLATES = {
        ArtifactType.SRS: [
            "{title} 요구사항",
            "{title} 기능 명세",
            "{title} 어떻게 구현",
        ],
        ArtifactType.HLD: [
            "{title} 아키텍처",
            "{title} 설계 구조",
            "{title} 시스템 구성",
        ],
        ArtifactType.LLD: [
            "{title} 상세 설계",
            "{title} 클래스 다이어그램",
            "{title} 시퀀스",
        ],
        ArtifactType.TEST_CASE: [
            "{title} 테스트 방법",
            "{title} 테스트 케이스",
            "{title} 검증",
        ],
        ArtifactType.API_SPEC: [
            "{title} API",
            "{title} 엔드포인트",
            "{title} 요청 응답",
        ],
    }
    
    @classmethod
    def enrich(cls, artifact: SIArtifact) -> SIArtifact:
        """산출물 컨텍스트 보강"""
        
        # 1. 키워드 추출
        if not artifact.keywords:
            artifact.keywords = cls._extract_keywords(artifact)
        
        # 2. 관련 질의 생성
        if not artifact.related_queries:
            artifact.related_queries = cls._generate_queries(artifact)
        
        # 3. 자동 태그
        artifact.tags = cls._generate_tags(artifact)
        
        # 4. 요약 생성 (LLM 호출 또는 규칙 기반)
        if not artifact.summary:
            artifact.summary = cls._generate_summary(artifact)
        
        return artifact
    
    @classmethod
    def _extract_keywords(cls, artifact: SIArtifact) -> list[str]:
        """키워드 추출"""
        import re
        from collections import Counter
        
        text = f"{artifact.title} {artifact.content}"
        
        # 한글 + 영어 + 숫자 조합 단어 추출
        words = re.findall(r'[가-힣]+|[a-zA-Z][a-zA-Z0-9_]*', text)
        
        # 불용어 제거
        stopwords = {'이', '그', '저', '것', '수', '등', '및', '의', '를', '을', '에', 
                     '가', '은', '는', '으로', '로', '와', '과', 'the', 'a', 'an', 'is', 
                     'are', 'be', 'to', 'of', 'and', 'or', 'in', 'for'}
        words = [w for w in words if w.lower() not in stopwords and len(w) > 1]
        
        counter = Counter(words)
        return [word for word, _ in counter.most_common(15)]
    
    @classmethod
    def _generate_queries(cls, artifact: SIArtifact) -> list[str]:
        """예상 질의 생성"""
        queries = []
        
        # 템플릿 기반 질의
        templates = cls.QUERY_TEMPLATES.get(artifact.artifact_type, [])
        for template in templates:
            queries.append(template.format(title=artifact.title))
        
        # 키워드 기반 질의
        for kw in artifact.keywords[:5]:
            queries.append(f"{kw} 관련 문서")
            queries.append(f"{kw} {artifact.artifact_type.value}")
        
        return queries
    
    @classmethod
    def _generate_tags(cls, artifact: SIArtifact) -> list[str]:
        """자동 태그 생성"""
        tags = [
            artifact.artifact_type.value,
            artifact.phase,
            artifact.status
        ]
        
        # 내용 기반 태그
        content_lower = artifact.content.lower()
        if any(word in content_lower for word in ['api', 'rest', 'endpoint']):
            tags.append("API")
        if any(word in content_lower for word in ['db', 'database', '데이터베이스', 'sql']):
            tags.append("DATABASE")
        if any(word in content_lower for word in ['ui', '화면', 'frontend', '프론트']):
            tags.append("FRONTEND")
        if any(word in content_lower for word in ['batch', '배치', 'scheduler']):
            tags.append("BATCH")
        
        return tags
    
    @classmethod
    def _generate_summary(cls, artifact: SIArtifact) -> str:
        """요약 생성"""
        # 간단한 규칙 기반 (실제로는 LLM 사용 권장)
        return f"[{artifact.artifact_type.value}] {artifact.title} - {artifact.phase} 단계"


# =============================================================================
# 6. 사용 예시
# =============================================================================

def demo():
    """사용 예시"""
    
    print("=" * 70)
    print("SI 프로젝트 GraphRAG 시스템")
    print("=" * 70)
    
    # 1. 산출물 생성 예시
    print("\n[1] 산출물 생성 및 컨텍스트 보강")
    print("-" * 50)
    
    # 요구사항 정의서
    srs = SIArtifact(
        _key="SRS-001",
        artifact_type=ArtifactType.SRS,
        title="사용자 인증 기능 요구사항",
        content="""
        1. 기능 개요
        사용자는 이메일/비밀번호로 로그인할 수 있어야 한다.
        
        2. 상세 요구사항
        - REQ-001: 이메일 형식 검증
        - REQ-002: 비밀번호 8자 이상, 특수문자 포함
        - REQ-003: 로그인 실패 5회 시 계정 잠금
        - REQ-004: JWT 토큰 기반 세션 관리
        
        3. 비기능 요구사항
        - 로그인 응답시간 2초 이내
        - 동시 접속 10,000명 지원
        """,
        project_code="PRJ-2024-001",
        phase="분석",
        author="홍길동"
    )
    
    # 컨텍스트 보강
    enriched_srs = ArtifactEnricher.enrich(srs)
    
    print(f"산출물: {enriched_srs._key}")
    print(f"키워드: {enriched_srs.keywords}")
    print(f"관련 질의: {enriched_srs.related_queries[:3]}")
    print(f"태그: {enriched_srs.tags}")
    
    # 설계서
    hld = SIArtifact(
        _key="HLD-001",
        artifact_type=ArtifactType.HLD,
        title="인증 모듈 고수준 설계",
        content="""
        1. 아키텍처 개요
        - 인증 서버는 별도 마이크로서비스로 분리
        - Redis를 활용한 세션 캐싱
        - OAuth2.0 / OIDC 지원
        
        2. 컴포넌트 구성
        - AuthController: 인증 API 엔드포인트
        - AuthService: 인증 비즈니스 로직
        - TokenProvider: JWT 토큰 발급/검증
        - UserRepository: 사용자 정보 조회
        """,
        project_code="PRJ-2024-001",
        phase="설계",
        author="김철수"
    )
    
    # 테스트 케이스
    tc = SIArtifact(
        _key="TC-001",
        artifact_type=ArtifactType.TEST_CASE,
        title="로그인 기능 테스트",
        content="""
        TC-001-01: 정상 로그인 테스트
        - 사전조건: 유효한 계정 존재
        - 입력: 올바른 이메일/비밀번호
        - 예상결과: JWT 토큰 반환, 200 OK
        
        TC-001-02: 비밀번호 오류 테스트
        - 입력: 잘못된 비밀번호
        - 예상결과: 401 Unauthorized
        
        TC-001-03: 계정 잠금 테스트
        - 사전조건: 로그인 4회 실패 상태
        - 입력: 잘못된 비밀번호
        - 예상결과: 계정 잠금, 423 Locked
        """,
        project_code="PRJ-2024-001",
        phase="테스트",
        author="이영희"
    )
    
    # 2. 관계 정의 예시
    print("\n[2] 산출물 간 관계 (추적성)")
    print("-" * 50)
    
    relations = [
        ArtifactRelation(
            _from="si_artifacts/HLD-001",
            _to="si_artifacts/SRS-001",
            relation_type=RelationType.IMPLEMENTS,
            description="인증 모듈 설계는 인증 요구사항을 구현",
            trace_id="TRACE-001"
        ),
        ArtifactRelation(
            _from="si_artifacts/TC-001",
            _to="si_artifacts/SRS-001",
            relation_type=RelationType.TESTS,
            description="로그인 테스트는 인증 요구사항을 검증",
            trace_id="TRACE-002"
        ),
    ]
    
    for rel in relations:
        print(f"  {rel._from.split('/')[-1]} --[{rel.relation_type.value}]--> {rel._to.split('/')[-1]}")
    
    # 3. 검색 쿼리 예시
    print("\n[3] GraphRAG 검색 쿼리 예시")
    print("-" * 50)
    
    # 하이브리드 검색
    print("\n### 하이브리드 검색 (벡터 + 전문검색)")
    query = SIGraphRAGQueries.hybrid_search(
        search_text="로그인 인증 JWT",
        embedding=[0.0] * 10,  # placeholder
        project_code="PRJ-2024-001",
        artifact_types=["SRS", "HLD"],
        limit=5
    )
    print("  검색어: '로그인 인증 JWT'")
    print("  필터: project=PRJ-2024-001, types=[SRS, HLD]")
    
    # 추적성 쿼리
    print("\n### 추적성 쿼리")
    trace_query = SIGraphRAGQueries.traceability_query(
        artifact_key="SRS-001",
        direction="forward"
    )
    print("  시작점: SRS-001 (요구사항)")
    print("  방향: forward (요구사항 → 설계 → 구현 → 테스트)")
    
    # 영향도 분석
    print("\n### 영향도 분석")
    impact_query = SIGraphRAGQueries.impact_analysis(artifact_key="SRS-001")
    print("  대상: SRS-001")
    print("  분석: 이 요구사항 변경 시 영향받는 설계서, 테스트케이스 등")
    
    # 컨텍스트 보강 검색
    print("\n### 컨텍스트 보강 검색")
    context_query = SIGraphRAGQueries.context_enriched_search(
        search_text="비밀번호 정책",
        depth=2
    )
    print("  검색어: '비밀번호 정책'")
    print("  결과: 검색 문서 + 상위/하위 관련 문서 + 연관 이슈")
    
    print("\n" + "=" * 70)
    print("GraphRAG 장점:")
    print("  1. 문서 간 추적성 자동 관리")
    print("  2. 변경 영향도 즉시 파악")
    print("  3. 컨텍스트 기반 정확한 검색")
    print("  4. LLM 응답에 풍부한 컨텍스트 제공")
    print("=" * 70)


if __name__ == "__main__":
    demo()